In [22]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import seaborn as sns
import tensorflow as tf

from matplotlib import pylab as plt
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from keras.layers.convolutional import Conv1D, MaxPooling1D
from sklearn.metrics import accuracy_score
from keras.layers import Dense,LSTM, BatchNormalization, Dropout, Bidirectional, Flatten, LeakyReLU, Conv3D, MaxPooling3D
from sklearn.model_selection import GridSearchCV


path = os.getcwd() + '/'
path_out = 'RESULTADOS_REDES/'

In [15]:
def nc4(datos_datos, name):

    from datetime import datetime, timedelta
    from netCDF4 import Dataset,num2date,date2num
        
    a = test_data_tem.strftime[0].values
    b = test_data_tem.strftime[len(test_data_tem.strftime)-1].values

    nyears = len(test_data_tem.strftime);
    unout = 'days since ' + str(pd.to_datetime(a).year) + '-' + str(pd.to_datetime(a).month) + '-' + str(pd.to_datetime(a).day)

    nx, ny = (len(test_data_tem.latitude), len(test_data_tem.longitude))

    lat = test_data_tem.latitude.values
    lon = test_data_tem.longitude.values

    dataout = datos_datos
        
    datesout = np.arange(datetime(int(pd.to_datetime(a).year),int(pd.to_datetime(a).month),int(pd.to_datetime(a).day)),
                        datetime(int((pd.to_datetime(b) + timedelta(days=1)).year),int((pd.to_datetime(b) + timedelta(days=1)).month),int((pd.to_datetime(b) + timedelta(days=1)).day)),
                        timedelta(days = 1)).astype(datetime)
        
    ncout = Dataset(path + path_out + name + '.nc','w','NETCDF3')
    ncout.createDimension('longitude',ny)
    ncout.createDimension('latitude',nx)
    ncout.createDimension('time',nyears)
    lonvar = ncout.createVariable('longitude','float32',('longitude'));lonvar[:] = lon
    latvar = ncout.createVariable('latitude','float32',('latitude'));latvar[:] = lat
    timevar = ncout.createVariable('time','float64',('time'));timevar.setncattr('units',unout);timevar[:]=date2num(datesout,unout)
    myvar = ncout.createVariable(name,'float32',('time','latitude','longitude'));myvar.setncattr('units','');myvar[:] = dataout
    ncout.close()

In [23]:
#modis      = pd.read_csv('DATA_SEMILLERO/INCENDIOS_ZONA.csv')
nc_org     = xr.open_dataset('DATA_DAY/2m_temperature.nc')-273.15
nc_tem_no     = xr.open_dataset('DATA_DAY/t2m_mod.nc')-273.15
nc_tem     = xr.open_dataset('DATA_TEM_MOD/t2m_mod.nc')-273.15
nc_rad     = xr.open_dataset('DATA_DAY/surface_solar_radiation_downwars.nc')
nc_soiltem = xr.open_dataset('DATA_DAY/soil_temperature_level_1.nc')-273.15
nc_pre     = xr.open_dataset('DATA_DAY/total_precipitation.nc')*1000
nc_dew     = xr.open_dataset('DATA_DAY/2m_dewpoint_temperature.nc')-273.15
nc_uwind   = xr.open_dataset('DATA_DAY/10m_u_component_of_wind.nc')
nc_vwind   = xr.open_dataset('DATA_DAY/10m_v_component_of_wind.nc')
nc_leafhigh= xr.open_dataset('DATA_DAY/leaf_area_index_high_vegetation.nc')
nc_leaflow = xr.open_dataset('DATA_DAY/leaf_area_index_low_vegetation.nc')
nc_skin    = xr.open_dataset('DATA_DAY/skin_reservoir_content.nc')*1000
nc_vol     = xr.open_dataset('DATA_DAY/volumetric_soil_water_layer_1.nc')
nc_eva     = xr.open_dataset('DATA_DAY/evaporation.nc')*(-1000)

nc_BUI  = xr.open_dataset('../../FWI/RESULTADOS_FWI10_NC/BUI.nc')
nc_DC   = xr.open_dataset('../../FWI/RESULTADOS_FWI10_NC/DC.nc')
nc_DMC  = xr.open_dataset('../../FWI/RESULTADOS_FWI10_NC/DMC.nc')
nc_DSR  = xr.open_dataset('../../FWI/RESULTADOS_FWI10_NC/DSR.nc')
nc_FFMC = xr.open_dataset('../../FWI/RESULTADOS_FWI10_NC/FFMC.nc')
nc_FWI  = xr.open_dataset('../../FWI/RESULTADOS_FWI10_NC/FWI.nc')
nc_ISI  = xr.open_dataset('../../FWI/RESULTADOS_FWI10_NC/ISI.nc')
nc_M    = xr.open_dataset('../../FWI/RESULTADOS_FWI10_NC/M.nc')
nc_WIND = xr.open_dataset('../../FWI/RESULTADOS_FWI10_NC/WIND.nc')
nc_RH   = xr.open_dataset('../../FWI/RESULTADOS_FWI10_NC/RH.nc')

nc_org = nc_org.rename({'t2m': 't2m_org'})

In [24]:
tem      = nc_tem.t2m.assign_coords(strftime = nc_tem.t2m.strftime.values.astype('datetime64[ns]'))  #DESACTIVAR con datos modificados en tiempo o long
pre      = nc_pre.tp.assign_coords(strftime = nc_pre.tp.strftime.values.astype('datetime64[ns]'))
rad      = nc_rad.ssrd.assign_coords(strftime = nc_rad.ssrd.strftime.values.astype('datetime64[ns]'))
soiltem  = nc_soiltem.stl1.assign_coords(strftime = nc_soiltem.stl1.strftime.values.astype('datetime64[ns]'))     
dew      = nc_dew.d2m.assign_coords(strftime = nc_dew.d2m.strftime.values.astype('datetime64[ns]'))    
uwind    = nc_uwind.u10.assign_coords(strftime = nc_uwind.u10.strftime.values.astype('datetime64[ns]'))   
vwind    = nc_vwind.v10.assign_coords(strftime = nc_vwind.v10.strftime.values.astype('datetime64[ns]'))  
leafhigh = nc_leafhigh.lai_hv.assign_coords(strftime = nc_leafhigh.lai_hv.strftime.values.astype('datetime64[ns]'))
leaflow  = nc_leaflow.lai_lv.assign_coords(strftime = nc_leaflow.lai_lv.strftime.values.astype('datetime64[ns]'))
skin     = nc_skin.src.assign_coords(strftime = nc_skin.src.strftime.values.astype('datetime64[ns]'))
vol      = nc_vol.swvl1.assign_coords(strftime = nc_vol.swvl1.strftime.values.astype('datetime64[ns]'))
eva      = nc_eva.e.assign_coords(strftime = nc_eva.e.strftime.values.astype('datetime64[ns]'))

nc_BUI  = nc_BUI.BUI.rename({'time':'strftime'}) 
nc_DC   = nc_DC.DC.rename({'time':'strftime'}) 
nc_DMC  = nc_DMC.DCM.rename({'time':'strftime'})
nc_DSR  = nc_DSR.DSR.rename({'time':'strftime'})
nc_FFMC = nc_FFMC.FFMC.rename({'time':'strftime'})
nc_FWI  = nc_FWI.FWI.rename({'time':'strftime'})
nc_ISI  = nc_ISI.ISI.rename({'time':'strftime'})
nc_M    = nc_M.M.rename({'time':'strftime'})
nc_WIND = nc_WIND.WIND.rename({'time':'strftime'})
nc_RH = nc_RH.RH.rename({'time':'strftime'})

BUI  = nc_BUI.assign_coords(strftime = nc_BUI.strftime.values.astype('datetime64[ns]'))  
DC   = nc_DC.assign_coords(strftime = nc_DC.strftime.values.astype('datetime64[ns]'))  
DMC  = nc_DMC.assign_coords(strftime = nc_DMC.strftime.values.astype('datetime64[ns]'))  
DSR  = nc_DSR.assign_coords(strftime = nc_DSR.strftime.values.astype('datetime64[ns]'))  
FFMC = nc_FFMC.assign_coords(strftime = nc_FFMC.strftime.values.astype('datetime64[ns]')) 
FWI  = nc_FWI.assign_coords(strftime = nc_FWI.strftime.values.astype('datetime64[ns]')) 
ISI  = nc_ISI.assign_coords(strftime = nc_ISI.strftime.values.astype('datetime64[ns]'))  
M    = nc_M.assign_coords(strftime = nc_M.strftime.values.astype('datetime64[ns]')) 
WIND = nc_WIND.assign_coords(strftime = nc_WIND.strftime.values.astype('datetime64[ns]'))
RH = nc_RH.assign_coords(strftime = nc_RH.strftime.values.astype('datetime64[ns]'))

max_tem = np.max(tem.values)
min_tem = np.min(tem.values)

In [25]:
def div(data_var):
    max_var = np.max(data_var.values)
    min_var = np.min(data_var.values)
    result_data_var = (data_var-min_var)/(max_var-min_var)
    result_data_var = np.round(result_data_var,4)
    return result_data_var

In [26]:
pre      = div(pre)
eva      = div(eva)
rad      = div(rad)
soiltem  = div(soiltem)
leafhigh = div(leafhigh)
skin     = div(skin)
vol      = div(vol)
tem      = div(tem)
dew      = div(dew)

DMC      = div(DMC)
DSR      = div(DSR)
ISI      = div(ISI)
DC       = div(DC)
WIND     = div(WIND)
RH       = div(RH)

In [31]:
Var_data = xr.concat([pre,eva,leafhigh,skin,soiltem,rad,DMC,DSR,WIND,tem], dim = 'nueva_dimension')
Var_data = Var_data.transpose('strftime','latitude','longitude','nueva_dimension')

contar = int(len(Var_data)*0.8)
train_data = Var_data[:contar,:,:,:]
test_data  = Var_data[contar:,:,:,:]

x_train = []
y_train = []

for kk in np.arange(0,len(train_data),1):
    x_train.append(train_data[kk:kk+1,:,:,:-1].values) # tiempo,lat, lon, numero var
    y_train.append(train_data[kk:kk+1,:,:,9].values)  

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1],x_train.shape[2],x_train.shape[3], x_train.shape[4]))
y_train = np.reshape(y_train, (y_train.shape[0], y_train.shape[1] * y_train.shape[2] * y_train.shape[3]))

x_test = []
y_test = []

for kk in np.arange(0,len(test_data),1):
    x_test.append(test_data[kk:kk+1,:,:,:-1].values) # tiempo,lat, lon, numero var
    y_test.append(test_data[kk:kk+1,:,:,9].values)
    
x_test, y_test = np.array(x_test), np.array(y_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1],x_test.shape[2],x_test.shape[3], x_test.shape[4]))
y_test = np.reshape(y_test, (y_test.shape[0], y_test.shape[1] * y_test.shape[2] * y_test.shape[3]))

In [8]:
model = Sequential()

model.add(Conv3D(filters = 64, kernel_size = (1,3,3)
                 input_shape=np.shape(x_train)[1:])) 

model.add(BatchNormalization()) 
model.add(Conv3D(filters = 128,kernel_size = (1,3,3)))
model.add(tf.keras.layers.ConvLSTM2D(filters = 128, kernel_size = (3,3)))
model.add(Flatten())

model.add(Dense(49*2, activation='relu'))
Dropout(0.6)
model.add(Dense(np.shape(y_train)[1], activation='relu'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 1, 5, 5, 64)       6976      
                                                                 
 batch_normalization (BatchN  (None, 1, 5, 5, 64)      256       
 ormalization)                                                   
                                                                 
 conv3d_1 (Conv3D)           (None, 1, 3, 3, 128)      73856     
                                                                 
 conv_lstm2d (ConvLSTM2D)    (None, 1, 1, 128)         1180160   
                                                                 
 flatten (Flatten)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 98)                12642     
                                                        

In [ ]:
model.compile(optimizer='Adam', loss='mse')

from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor = 'val_loss', patience = 20, restore_best_weights = True)
historia = model.fit(x_train, y_train,epochs = 100,verbose = 1, validation_data = (x_test, y_test))

In [249]:
#model.save(path + 'MODELOS3D/MODEL_C2.h5')
#model.save_weights(path + 'MODELOS3D/PESOS_MODEL_C2.h5')

In [10]:
prediccion = model.predict(x_test)
prediccion = np.reshape(prediccion, (prediccion.shape[0], 7, 7)); prediccion = (prediccion * (max_tem - min_tem)) + min_tem

14/14 [==============================] - 0s 4ms/step


In [12]:
contar2 = int(len(tem)*0.8)
test_data_tem = tem[contar2:len(tem),:,:]

In [13]:
#nc4(prediccion, 't2m_predC3')
real = np.reshape(y_test, (y_test.shape[0], 7, 7)); real = (real * (max_tem - min_tem)) + min_tem
#nc4(real, 't2m_realC')